In [1]:
import pandas as pd
import numpy as np
import glob
import nltk
import re
import heapq
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktLanguageVars
import pickle
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

### Loading pre-trained model

In [3]:
loaded_vec= CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb"))) # your path here 
loaded_tfidf = pickle.load(open("tfidf.pkl", "rb")) # your path here
loaded_model = pickle.load(open("nb_model.pkl", "rb")) # your path here

### Loading testing data

In [4]:
X_test = pickle.load(open("x_test.pkl", "rb")) # your path here
Y_test = pickle.load(open("y_test.pkl", "rb")) # your path here

### Model Usage / Classification

In [5]:
category_list = ['business','crime','editorial','entertainment','politic']
predicted = loaded_model.predict(X_test)

print("Predicted Label - Actual Label")
print("------------------------------")
for predicted_item, result in zip(predicted, Y_test):
    print(category_list[predicted_item], ' - ' , category_list[result])

Predicted Label - Actual Label
------------------------------
politic  -  politic
business  -  business
editorial  -  editorial
crime  -  crime
editorial  -  editorial
politic  -  politic
crime  -  crime
business  -  business
entertainment  -  entertainment
business  -  business
crime  -  crime
editorial  -  editorial
entertainment  -  entertainment
crime  -  crime
crime  -  crime
politic  -  politic
politic  -  politic
editorial  -  business
crime  -  crime
crime  -  politic
editorial  -  editorial
editorial  -  editorial
politic  -  politic
business  -  business
business  -  business
business  -  business
entertainment  -  entertainment
politic  -  politic
business  -  business
business  -  business
politic  -  politic
business  -  business
crime  -  crime
business  -  business
entertainment  -  entertainment
business  -  business
editorial  -  editorial
editorial  -  crime
business  -  politic
editorial  -  editorial
editorial  -  crime
politic  -  politic
business  -  business
ente

### Confusion Matrix

In [6]:
print(pd.DataFrame(confusion_matrix(Y_test, predicted), index=category_list, columns=category_list))

               business  crime  editorial  entertainment  politic
business             28      0          1              0        2
crime                 0     16          2              0        0
editorial             0      0         23              0        0
entertainment         0      0          1             20        0
politic               1      1          2              0       28


### Printing accuracy score

In [7]:
accuracy_score(Y_test, predicted)

0.92

### Testing another news article

In [9]:
doc_news = """ ယမန်နေ့က လော့စ်အိန်ဂျလိစ်မြို့၊ Beverly Hilton ဟိုတယ်မှာ အော်စကာဆုလျာထားခံရတဲ့ ရုပ်ရှင်မင်းသား၊ မင်းသမီးတွေ စုရုံးပြီး နေ့လယ်စာ အတူတကွသုံးဆောင်ခဲ့ကြတယ်လို့ သိရပါတယ်။
အဆိုပါနေ့လယ်စာစားပွဲကို ရုပ်ရှင်မင်းသမီး၊ အဆိုတော် Lady Gaga၊ ရုပ်ရှင်မင်းသား၊ ဒါရိုက်တာ Bradley Cooper၊ ရုပ်ရှင်မင်းသမီး Glenn Close၊ ရုပ်ရှင်မင်းသား Viggo Mortensen၊ ရုပ်ရှင်ဒါရိုက်တာ Spike Lee၊ ရုပ်ရှင်မင်းသမီး Melissa McCarthy အပါအဝင် ကမ္ဘာတစ်ဝန်းက အော်စကာဆုလျာထားခံရတဲ့ အခြားအခြားသော အနုပညာရှင်တွေ တက်ရောက်ခဲ့ကြတာဖြစ်ပါတယ်။
အကောင်းဆုံး နိုင်ငံခြားရုပ်ရှင်ဇာတ်ကားဆုလျာထားခံရတဲ့ ဇာတ်ကားတွေအနက် Netflix ရုပ်သံလိုင်းမှာ ရေပန်းစားနေတဲ့ Roma မက္ကဆီကိုဇာတ်ကား၊ The Favourite ဗြိတိသျှဇာတ်ကား၊ Cold War ပိုလန်ဇာတ်ကား၊ Never Look Away ဂျာမနီဇာတ်ကားနဲ့ Shoplifters ဂျပန်ဇာတ်ကားတို့ ပါဝင်ကြောင်း သိရှိရပါတယ်။
ပြီးခဲ့တဲ့လအနည်းငယ်က ရုံတင်ပြသခဲ့တဲ့ Black Panther၊ A Star is Born ရုပ်ရှင်ကားတွေအပြင် BlacKkKlansman၊ Bohemian Rhapsody၊ La Favorite၊ Green Book အစရှိတဲ့ ဇာတ်ကားတွေကတော့ အကောင်းဆုံးရုပ်ရှင်ဇာတ်ကားဆု လျာထားခံရတဲ့ ဇာတ်ကားတွေဖြစ်ပါတယ်။
အကောင်းဆုံး အမျိုးသားရုပ်ရှင်ဇာတ်ဆောင်ဆု လျာထားခံရသူတွေထဲမှာ Vice ဇာတ်ကားထဲမှာ သရုပ်ဆောင်ထားတဲ့ မင်းသား Christian Bale၊ A Star is Born ဇာတ်ကားရဲ့ ဇာတ်ဆောင် Bradley Cooper၊ At Eternity’s Gate ဇာတ်ကားမှာ ပါဝင်သရုပ်ဆောင်ထားတဲ့ မင်းသား Willem Dafoe၊ Bohemian Rhapsody ဇာတ်ကားရဲ့ ဇာတ်ဆောင်မင်းသား Rami Malek နဲ့ Green Book ဇာတ်ကားမှာ သရုပ်ဆောင်ထားတဲ့ Viggo Mortensen တို့ပါဝင်ပါတယ်။
အကောင်းဆုံး အမျိုးသမီးရုပ်ရှင်ဇာတ်ဆောင်ဆု လျာထားခံရသူတွေထဲမှာတော့ Roma ဇာတ်ကားရဲ့ အမျိုးသမီးဇာတ်ဆောင် Yalitza Aparicio၊ A Star is Born ရုပ်ရှင်ကားရဲ့ အမျိုးသမီးဇာတ်ဆောင် Lady Gaga၊ Can You Ever Forgive Me ဇာတ်ကားမှာ သရုပ်ဆောင်ထားတဲ့ Melissa McCarthy၊ The Wife ဇာတ်ကားရဲ့ အမျိုးသမီးဇာတ်ဆောင် Glenn Close တို့ပါဝင်ပါတယ်။
ရုပ်ရှင်လောကရဲ့ အထင်ကရဆုပေးပွဲကြီးဖြစ်တဲ့ ၉၁ ကြိမ်မြောက် အော်စကာဆုပေးပွဲကို လာမယ့် ဖေဖော်ဝါရီလ၊ ၂၅ ရက်၊ Dolby ပြဇာတ်ရုံ၊ လော့စ်အိန်ဂျလိစ်မြို့မှာ ကျင်းပပြုလုပ်သွားမှာဖြစ်ပါတယ်။"""

doc_news = [doc_news]

vector = loaded_vec.transform(doc_news)
tfidf = loaded_tfidf.transform(vector)
predicted = loaded_model.predict(tfidf)
print(category_list[predicted[0]])

entertainment
